In [6]:
import pandas as pd
import protfasta
import re
from os import listdir
from os.path import isfile, join

Next cell code from http://localhost:8888/notebooks/Desktop/Staller_Lab/SFARI/soto_analysis/notebooks/Variants%20to%20predictor%20input.ipynb

In [9]:
from os import listdir
from os.path import isfile, join
#https://stackoverflow.com/questions/3207219/how-do-i-list-all-files-of-a-directory
ADs = [f for f in listdir("../soto_analysis/outputs/AD_variant_fasta/") if isfile(join("../soto_analysis/outputs/AD_variant_fasta/", f))]

def return_AD_df(AD):
    if "KMT2A-KMT2A-ALL1-HRX-KMT2A-MLL1-CXXC7" not in AD: #There is an issue with the coordinates for this, so skipping for now
        AD_dict = protfasta.read_fasta("../soto_analysis/outputs/AD_variant_fasta/" + AD)
        df = pd.DataFrame({"name" : AD_dict.keys(), "seq" : AD_dict.values()})
        df["name"] = AD + "|" + df["name"]
        return df

dfs_to_concat = []
for AD in ADs:
    dfs_to_concat.append(return_AD_df(AD))

all_variant_ADs = pd.concat(dfs_to_concat)
all_variant_ADs

,name,seq
0,GLIS1_Q8NBF1_AD_447-620|g.53506627C>T(p.Asp619...,AESTRDGLGPGLLSPIVSPLKGLGPPPLPPSSQSHSPGGQPFPTLP...
1,GLIS1_Q8NBF1_AD_447-620|g.53506630T>C(p.Thr618...,AESTRDGLGPGLLSPIVSPLKGLGPPPLPPSSQSHSPGGQPFPTLP...
2,GLIS1_Q8NBF1_AD_447-620|g.53506633A>G(p.Tyr617...,AESTRDGLGPGLLSPIVSPLKGLGPPPLPPSSQSHSPGGQPFPTLP...
3,GLIS1_Q8NBF1_AD_447-620|g.53506638G>A(p.Ser615...,AESTRDGLGPGLLSPIVSPLKGLGPPPLPPSSQSHSPGGQPFPTLP...
4,GLIS1_Q8NBF1_AD_447-620|g.53506641G>A(p.Pro614...,AESTRDGLGPGLLSPIVSPLKGLGPPPLPPSSQSHSPGGQPFPTLP...
...,...,...
35,TFE3_P19532_AD_472-575|g.49030459G>A(p.Thr476Met),PGAAMFHVGGGPAQNAPHQQPPAPPSDALLDLHFPSDHLGDLGDPF...
36,TFE3_P19532_AD_472-575|g.49030465G>A(p.Ala474Val),PGVATFHVGGGPAQNAPHQQPPAPPSDALLDLHFPSDHLGDLGDPF...
37,TFE3_P19532_AD_472-575|g.49030466C>T(p.Ala474Thr),PGTATFHVGGGPAQNAPHQQPPAPPSDALLDLHFPSDHLGDLGDPF...
38,TFE3_P19532_AD_472-575|g.49030468C>T(p.Gly473Asp),PDAATFHVGGGPAQNAPHQQPPAPPSDALLDLHFPSDHLGDLGDPF...


In [123]:
changes = all_variant_ADs["name"].str.extract(r'p.([a-zA-Z]{3})\d+([a-zA-Z]{3})')
changes[0] = changes[0].str.upper()
changes[1] = changes[1].str.upper()
changes = changes.rename(columns = {0 : "orig", 1 : "var"})
changes = changes.reset_index(drop = True)
changes

,orig,var
0,ASP,ASN
1,THR,ALA
2,TYR,HIS
3,SER,PHE
4,PRO,LEU
...,...,...
3152,THR,MET
3153,ALA,VAL
3154,ALA,THR
3155,GLY,ASP


In [126]:
changes["orig"].value_counts()

PRO    451
ARG    341
SER    312
ALA    263
GLY    240
THR    200
ASP    167
GLU    167
VAL    151
LEU    143
GLN    133
MET    132
ASN    124
ILE    100
HIS     88
TYR     55
LYS     37
PHE     26
CYS     23
TRP      4
Name: orig, dtype: int64

In [127]:
changes["var"].value_counts()

SER    347
LEU    291
THR    238
ARG    231
VAL    231
ALA    202
ILE    158
HIS    157
GLY    155
GLN    149
CYS    136
LYS    122
PRO    117
ASN    114
GLU    106
ASP    103
MET     90
PHE     87
TYR     67
TRP     56
Name: var, dtype: int64

In [128]:
#https://stackoverflow.com/questions/12760271/how-do-i-convert-the-three-letter-amino-acid-codes-to-one-letter-code-with-pytho
d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

In [129]:
changes["orig"] = changes["orig"].map(d)
changes["var"] = changes["var"].map(d)

In [130]:
changes

,orig,var
0,D,N
1,T,A
2,Y,H
3,S,F
4,P,L
...,...,...
3152,T,M
3153,A,V
3154,A,T
3155,G,D


In [131]:
changes["orig"].value_counts()

P    451
R    341
S    312
A    263
G    240
T    200
D    167
E    167
V    151
L    143
Q    133
M    132
N    124
I    100
H     88
Y     55
K     37
F     26
C     23
W      4
Name: orig, dtype: int64

In [134]:
changes["var"].value_counts()

S    347
L    291
T    238
R    231
V    231
A    202
I    158
H    157
G    155
Q    149
C    136
K    122
P    117
N    114
E    106
D    103
M     90
F     87
Y     67
W     56
Name: var, dtype: int64

In [136]:
acidic = ["D", "E"]
basic = ["R", "K"]
aromatic = ["W", "F", "Y"]

In [137]:
orig_acidic = changes[changes["orig"].isin(acidic)]
orig_acidic["var"].value_counts()

K    59
G    59
N    46
E    44
D    41
V    24
H    17
Q    17
A    15
Y    12
Name: var, dtype: int64

In [147]:
orig_acidic_vars = pd.DataFrame(orig_acidic["var"].value_counts()).reset_index()
orig_acidic_vars

,index,var
0,K,59
1,G,59
2,N,46
3,E,44
4,D,41
5,V,24
6,H,17
7,Q,17
8,A,15
9,Y,12


In [148]:
orig_acidic_vars["f_obs"] = orig_acidic_vars["var"] / sum(orig_acidic_vars["var"])
orig_acidic_vars

,index,var,f_obs
0,K,59,0.176647
1,G,59,0.176647
2,N,46,0.137725
3,E,44,0.131737
4,D,41,0.122754
5,V,24,0.071856
6,H,17,0.050898
7,Q,17,0.050898
8,A,15,0.044910
9,Y,12,0.035928


In [138]:
orig_basic = changes[changes["orig"].isin(basic)]
orig_basic["var"].value_counts()

Q    90
H    61
C    58
W    45
G    27
L    18
K    18
R    13
E    12
P    10
T     9
S     8
I     4
M     3
N     2
Name: var, dtype: int64

In [139]:
orig_aromatic = changes[changes["orig"].isin(aromatic)]
orig_aromatic["var"].value_counts()

C    35
H    12
L    12
S     8
F     5
N     4
I     3
Y     2
V     1
R     1
G     1
D     1
Name: var, dtype: int64